In [ ]:
import os

import numpy as np
import cv2 
import matplotlib.pyplot as plt
import tensorflow 
import keras
import random

from sklearn.metrics import confusion_matrix
#print(np.array(y[0],dtype = np.uint8))
import sklearn.metrics


In [ ]:
from tensorflow.keras.layers import Input 
from tensorflow.keras.layers import Activation,Flatten,Reshape,Dense,Lambda,Dropout
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,UpSampling2D,Conv2D,SeparableConvolution2D,DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
#from tensorflow.keras import np_utils
#from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Concatenate,Add,Multiply
from tensorflow.keras import backend as K

In [ ]:
def down_block(x,filters,kernel_size=(3,3), padding="same",strides = 1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = MaxPooling2D((2, 2), strides=(2, 2),padding="same")(c)
    p = Dropout(0.2)(p)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = UpSampling2D((2, 2))(x)
    concat = Concatenate()([us, skip])
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    c = Dropout(0.2)(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    c = Dropout(0.2)(c)
    return c

In [ ]:
def Unet():
    f = [64, 128, 256,512,1024]
    inputs = Input((image_size, image_size,3))
    #print(inputs.shape)
    #inputs = np.expand_dims(inputs, axis=-1)
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    #nputs = np.expand_dims(inputs, axis=-1)
    #outputs = np.expand_dims(outputs, axis=-1)
    model = Model(inputs, outputs)
    return model

In [ ]:
model_Unet = Unet()
adam = optimizers.Adam(lr=0.0001 , decay=5*1e-7)
model_Unet.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model_Unet.summary()